+ Data Analyst Nanodegree
+ Project 3 - Data Wrangle OpenStreetMaps Data
+ Ricardo Yoshitomi

# Data Wrangling the OpenStreetMap
***

## Map Area

***

The objective of this project is to use data munging techniques to clean and audit the OpenStreetMap data for a chosen part of the world. The OpenStreetMap area that I've chosen is the city of Santo André, SP located in Brazil. Since Santo André is not in the 200 most popular cities in the world, I downloaded the OSM dataset in XML format of a pre-selected area within the region of São Paulo that I obtained by searching in the custom extract field.

+ https://www.openstreetmap.org/relation/298086
+ https://mapzen.com/data/metro-extracts/metro/sao-paulo_brazil/

I've chosen this map because it is my hometown and I am very curious to see some facts that I could not discover only by walking around the city. Santo André is a city located in the Metropolitan Region of São Paulo and it is part of a group of municipalities known as Greater ABC Region (<em>Região do Grande ABC</em>). All the codes and information I used in this project I found in [OSM XML - wiki](http://wiki.openstreetmap.org/wiki/OSM_XML), [Regular Expression HOWTO](https://docs.python.org/3/howto/regex.html), [w3schools](https://www.w3schools.com/sql/default.asp) and [stackoverflow](https://stackoverflow.com/) pages. 

## OpenStreetMap Data Format

***

The OpenStreetMap data is storage in XML format. Basically it is a list of instances of three different core elements (also known as <em>data primitives</em>):

+ <strong>Node</strong> is a point with a geographic position. It is defined by its coordinates (pairs of a latitude and a longitude) and node id. Nodes are often used to define the shape or "path" of a way, but can be used to define standalone point features. When used to define point features, a node will normally have at least one tag to define its purpose. For example, a restaurant may be tagged with `amenity=restaurant`.
+ <strong>Way</strong> is an ordered list of nodes. It can be represented by a polyline if the way is open or a polygon if they form a closed loop. An open way describes a linear feature such as streets, roads, streams and railway lines. A closed way is a way where the last node is shared with the first node forming a closed loop. A closed way can be interpreted as an area, examples of areas include: 
    + `leisure=park` to define the perimeter of a park
    + `amenity=school` to define the outline of a school
+ <strong>Relation</strong> is ordered list of one or more nodes, ways and/or relations as members which is used to define logical or geographic relationships between other elements. A member of a relation can optionally have a role which describes the part that a particular feature plays within a relation.

All types of data primitives (nodes, ways and relations) can have a tag to describe a specific feature:

+ <strong>Tag</strong> consists of two free format text fields, a key and a value separated by an equals sign, `key = value`. For example, `highway=residential` defines the way as a road whose main function is to give access to people's homes.

## Problems Encontered in the Data
***
After converting the XML file into CSV format using the data.py script and importing it into a SQL database in order to perform some queries, I identified some problems in the data that will be discussed in the following.

### Overabbreviated Street Names

First, I performed some basic querying in SQL to identify which type of street name abbreviations I could encounter. After that, I used regular expressions to identify specific cases more precisely. With the problems revealed, I iterated over each street word to correct them. The problems that I encountered are presented below:

+ "Av. Andrômeda" : Abbreviated street type. Correct: "Avenida Andrômeda"
+ "Avenida Sen. Vergueiro" : Abbreviated street name. Correct: "Avenida Senador Vergueiro"
+ "1ª Travessa da Estrada do Morro Grande" : Contractions in ordinal indicators. Correct: "Primeira Travessa da Estrada do Morro Grande"

The following code updates all the inconsistent substrings.

``` python
(...)

street_type_re = re.compile(r'\b\w+\S+\.?', re.IGNORECASE)
abbrev_street_re = re.compile(r'(\b\w+|_)[.]', re.IGNORECASE)
num_street_re = re.compile(r'\d[°ª]', re.IGNORECASE)
lowercase_re = re.compile(r'\b[a-z]+', re.LOCALE)
postalcode_re = re.compile(r'\d{5}[-]\d{3}')


expected = ["Rua", "Avenida", u"Praça", "Travessa", "Alameda", "Largo", "Rodovia", "Complexo", "Estrada", "Rodoanel", "Passagem"]

mapping = { "Av.": "Avenida",
            "Sen.": "Senador",
            "Prof.": "Professor",
            "Dr.": "Doutor",
            "Pres.": "Presidente",
            "B.": "Barreto",
            u"1ª": "Primeira"
          }

allowed_lowercase = ["da", "de", "do", "das", "dos", "e"]
         
(...)

def update_name(name, mapping):
    words_name = name.split(" ")
    if words_name not in expected:
        for word in words_name:
            if word in mapping:
                name = name.replace(word, mapping[word])
                
(...)
```

### Street Names without Street Types

There are some addresses strings missing the street type (Rua, Avenida, Travessa, etc.). For this case, I searched for each incomplete address on Google Map to see which type of street I should add in the string. 

+ Correct "Fernando Espírito Santo Alves de Mattos" to "Avenida Fernando Espírito Santo Alves de Mattos"
+ Correct "Garcia Lorca" to "Rua Garcia Lorca"

The following code adds a new "street type" to the beginning of the address string.

``` python
(...)

if words_name[0] not in expected:
    if words_name[0] not in mapping:
        if words_name[0] == "Fernando":
            name = "Avenida " + name
        elif words_name[0] == "rua":
            pass
        else:
            name = "Rua " + name           

(...)
```

### Street Name Initialized with Lowercase

Some of the words in street names are not capitalized:

+ "avenida Sapopemba" : Street type is not capitalized. Correct: "Avenida Sapopemba"
+ "Rua alexandre Galera" : Street name is not capitalized. Correct: "Rua Alexandre Galera"

The code below iterates over each name identifying the lowercase word and replaces it with the same capitalized word. The words "da", "de", "do", "das", "dos" and "e" are allowed within names in portuguese language. These words are the exceptions to the correction rule.

``` python
(...)

if word == word.lower():
    if word not in allowed_lowercase:
        name = name.replace(word, word.capitalize())            

(...)
```

### Inconsistent Postal Codes

The Postal Code format in Brazil (known as CEP - <em>Código de Endereçamento Postal</em>) is composed by 8 digits. These digits are split into 2 parts separated by a hyphen. The first part is composed by 5 digits and the second by 3. The structure of the "CEP" has the format XXXXX-XXX. After performing some basic queries in SQL I identified the following sort of problems in the dataset:

+ 097912-60 :  Hyphen positioned in the wrong place. Correct: 09791-260
+ 09060000 : Digits without hyphen. Correct: 09060-000
+ 09832 400 : Space instead of hyphen. Correct: 09832-400
+ 09790 - 400 : Spaces separating the hyphen from the digits. Correct: 09790-400

The following code updated the inconsistent postal codes:

``` python
(...)

def update_postalcode(code):
    digits = []
    for num in code:
        if num not in ["-", " "]:
            digits.append(num)
    
    code = "".join(digits[0:5])+"-"+"".join(digits[5:])
    
    return code

(...)
```

## Extending the Borders
***
Since the area of Santo André is not delimited by its borders, but actually, it is represented by a square in the Mapzen website, areas of surrounding cities are included within the dataset. In addition, I had to extend the border (square) when I was extracting the OSM XML file from the website because the area of Santo André is not large enough to attend the criteria of at least 50 MB size.

```sql
sqlite> SELECT tags.value, COUNT(*) as count 
FROM (SELECT * FROM nodes_tags UNION ALL 
      SELECT * FROM ways_tags) tags
WHERE tags.key LIKE '%city'
GROUP BY tags.value
ORDER BY count DESC
LIMIT 6;
```

```html
São Bernardo do Campo   2218
Santo André             296
São Caetano do Sul      165
São Paulo               163
Diadema                 145
Mauá                    87
```

From the results of the query, we can see that most part of the data belongs to São Bernardo do Campo which is a neighboring city of Santo André. It means that the extracted part of São Bernardo is much larger than the entire city of Santo André. All the cities in the results except São Paulo compose the Greater Region of ABC.

### Postal Codes out of range

The [Correios](http://www.buscacep.correios.com.br/sistemas/buscacep/buscaFaixaCep.cfm) (Brazilian Mail Service) provides the CEP range of all cities in Brazil. We are most interested in the first 5 digits because it identifies the region of the city. We can see from the results bellow that the region of Greater ABC is identified by the digits 09XXX-XXX.

```html
São Paulo                 01000-001 to 05999-999
São Paulo                 08000-000 to 08499-999
Santo André               09000-001 to 09299-999
Mauá                      09300-001 to 09399-999
Ribeirão Pires            09400-001 to 09449-999
São Caetano do Sul        09500-001 to 09599-999
São Bernardo do Campo     09600-001 to 09899-999
Diadema                   09900-001 to 09999-999
```

```sql
SELECT tags.value 
FROM (SELECT * FROM nodes_tags WHERE key='postcode' 
      UNION ALL 
      SELECT * FROM ways_tags WHERE key='postcode') tags
WHERE NOT tags.value LIKE '09%' 
AND tags.value NOT BETWEEN '01000%' AND '05999%' 
AND tags.value NOT BETWEEN '08000%' AND '08499%';
```

```html
11060-301
13087-901
```

There are two CEPs in the dataset that are out of range of valid CEPs for the analyzed region. The CEP: 11060-301 corresponds to the region of Santos/SP and CEP: 13087-901 to the region of Campinas/SP. I searched the incorrect CEPs in the dataset and by the street names I changed them to the correct one. 
+ 11060-301 : Avenida Padre Anchieta, 21 - São Bernardo do Campo/SP. Correct: 09891-420
+ 13087-901 : Avenida Dom Pedro II, 2112 - Santo André/SP. Correct: 09080-001

The following code updated the incorrect postal code:

``` python
(...)

def update_postalcode(code):
    correct_cep = {"11060-301":"09891-420", "13087-901": "09080-001"}
    if code in correct_cep:
        code = correct_cep[code]
    
    return code

(...)
```

## Data Overview
***
This section is an overview about the dataset, the SQL queries were used to gather the basic statistics and some information about the data.

### File sizes

```html
santo-andre_brazil.osm ........ 55.5 MB
santo-andre.db ................ 31.8 MB
nodes.csv ..................... 20.3 MB
nodes_tags.csv ................ 0.66 MB
ways.csv ...................... 2.5 MB
ways_tags.csv ................. 4.7 MB
ways_nodes.cv ................. 7.8 MB
```

### Number of nodes

```sql
sqlite> SELECT COUNT(*) FROM nodes;
```

238699

### Number of ways

```sql
sqlite> SELECT COUNT(*) FROM ways;
```

40858

### Number of unique users

```sql
sqlite> SELECT COUNT(DISTINCT(e.uid))          
FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) e;
```

397

### Top 10 contributing users

```sql
sqlite> SELECT e.user, COUNT(*) as num
FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e
GROUP BY e.user
ORDER BY num DESC
LIMIT 10;
```

```sql
AjBelnuovo          59812
Bonix-Mapper        52649
felipeacsi          17484
poschon             14621
MCPicoli            14377
cxs                 10321
StefanSP            9452
patodiez            8809
naoliv              7275
Rub21               5962
```

### Number of contributions per year

```sql
sqlite> SELECT DATE(e.timestamp,'start of year') as YEAR, COUNT(*)
FROM (SELECT timestamp FROM nodes UNION ALL SELECT timestamp FROM ways) e
GROUP BY YEAR
ORDER BY YEAR;
```

```sql
2007-01-01   4228
2008-01-01   1551
2009-01-01   234
2010-01-01   2689
2011-01-01   28321
2012-01-01   33803
2013-01-01   31150
2014-01-01   47705
2015-01-01   23446
2016-01-01   45201
2017-01-01   61229
```

## Additional Ideas
***

### Top 10 appearing amenities

```sql
sqlite> SELECT e.value, COUNT(*) as num
FROM (SELECT key, value FROM nodes_tags UNION ALL SELECT key, value FROM ways_tags) e
WHERE e.key='amenity'
GROUP BY e.value
ORDER BY num DESC
LIMIT 10;
```

```sql
parking               350
school                250
fuel                  236
restaurant            187
place_of_worship      181
bank                  145
pharmacy              94
fast_food             83
bench                 75
telephone             71
```

### Most popular fast food

```sql
sqlite> SELECT e.value, COUNT(*) as num
FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) e
    JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='fast_food' UNION ALL SELECT DISTINCT(id) FROM ways_tags WHERE value='fast_food') i
    ON e.id=i.id
WHERE e.key='name'
GROUP BY e.value
ORDER BY num DESC
LIMIT 3;
```

```html
McDonald's     13
Subway         11
Burger King    3
```

![Subway](img/subway.jpg)

Although the result of the query indicates that McDonald's is the most popular fast food, actually Subway has the greatest number of franchises. According to [Diário do Grande ABC](http://www.dgabc.com.br/noticia/1939499/subway-ultrapassa-mcdonald-s-na-regiao) (regional newspaper of Greater ABC), Subway overcame McDonald's in the region. The difference in the results occurred probably because the dataset is not counting the number of franchises inside buildings such as Shopping Centers, and the OSM XML file only covers the region of Santo André and some part of neighboring cities (not the whole city).


### Number of residential highways

```sql
sqlite> SELECT ways_tags.value, COUNT(*)
FROM ways_tags
    JOIN (SELECT DISTINCT(id) FROM ways_tags WHERE value='residential') i
    ON ways_tags.id=i.id
WHERE ways_tags.key='highway';
```

```sql
residential   16924
```

### Total number of highways

```sql
sqlite> SELECT key, COUNT(*)
FROM ways_tags
WHERE key='highway';
```

```sql
highway   27501
```

### Percentage of residential highways

We can see from the results of the queries that 61.4% (16924/27501) of the highways are classified as residential, more than a half of the highways serve as an access to housing. It means that the region lacks expressways. The borders of each city confuse to one another and we don't know when a city begins and finishes. There are few highways that link the cities and the connection between the Greater ABC region and São Paulo is narrow. A consequence of this structure is that the traffic in the region is getting worse as the population of the region increases. Depending on the time of the day (rush hour), the traffic can reach some parts of residential highways. 

### Are cycleway highways an alternative?

```sql
sqlite> SELECT ways_tags.value, COUNT(*)
FROM ways_tags
    JOIN (SELECT DISTINCT(id) FROM ways_tags WHERE value='cycleway') i
    ON ways_tags.id=i.id
WHERE ways_tags.key='highway';
```

```sql
cycleway   20
```

There are lots of plans aimed to reduce the traffic in the region, one of these plans is the construction of cycleways. Encouraging people to let their cars at home to go to their work with a bicycle is a good practice. In addition to the contribution to the reduction of traffic congestion, it is also cheap, environmentally friendly and healthy. The map bellow indicates the cycleways in the Greater ABC region, we can see from the result of the query that 20 is the number of cycleways in the region. It is still a low number compared to the total number of highways, in terms of percentage, it is only 0.000727% (20/27501). 

There are plans to increase the number of cycleways in the city of São Paulo until the year of 2030. According to [Diário do Grande ABC](http://www.dgabc.com.br/Noticia/1936437/plano-preve-1-7-mil-km-de-ciclovias-em-sao-paulo-ate-2030), the city plans to have 1,7 thousand of kilometers of cycleways until the year of 2030. The extension of cycleways, today the city has 498,3 km ([CEPSP - Companhia de Engenharia de Tráfego](http://www.cetsp.com.br/consultas/bicicleta/mapa-de-infraestrutura-cicloviaria.aspx)), aims to connect the remote highways to the downtown. Maybe with this fact, the Greater ABC region will also have a larger number of cycleways in the future.

![Cycleway](img/cycleway.jpg)

## Suggestions for Improvement
***

### Amenities with street name

```sql
sqlite> SELECT COUNT(*)
FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) e
    JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE key='amenity' UNION ALL SELECT DISTINCT(id) FROM ways_tags WHERE key='amenity') i
    ON e.id=i.id
WHERE key='street';
```

```sql
1423
```

### Amenities with house number

```sql
sqlite> SELECT COUNT(*)
FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) e
    JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE key='amenity' UNION ALL SELECT DISTINCT(id) FROM ways_tags WHERE key='amenity') i
    ON e.id=i.id
WHERE key='housenumber';
```

```sql
883
```

### Amenities with street name and house number

```sql
sqlite> SELECT COUNT(*)
FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) e
    JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE key='amenity' UNION ALL SELECT DISTINCT(id) FROM ways_tags WHERE key='amenity') i
    ON e.id=i.id
    JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE key='street' UNION ALL SELECT DISTINCT(id) FROM ways_tags WHERE key='street') o
    ON i.id=o.id
WHERE key='housenumber';
```

```sql
879
```

### Total number of amenities

```sql
sqlite> SELECT COUNT(*)
FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) e
    JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE key='amenity' UNION ALL SELECT DISTINCT(id) FROM ways_tags WHERE key='amenity') i
    ON e.id=i.id
WHERE key='amenity';
```

```sql
2325
```

Based on the results of the queries, only 37.8% (879/2325) of the amenities have information about their location (street name and house number). It would be interesting improve this percentage to users have access to more detailed information when consulting amenities in the OpenStreetMap dataset. Although amenities have the geographic position (latitude and longitude), this kind of information is not suitable for users to get the location of a place. Not only the address is a useful information, but also the phone number, opening hours, wheelchair access, etc. A way to deal with this problem is to getting access to datasets that provide description about the location of amenities and programmatically update the missing addresses. However, there could be some inconveniences when performing this task: 
+ Finding a reliable dataset to avoid importing wrong data
+ The conflict of the amenity name between the two datasets

## Conclusion

In this project, I used data cleaning techniques to audit the OpenStreetMap dataset of the region of Santo André. After converting the OSM XML file into CSV format and importing it into a SQL database, I performed some queries to identify some problems in the dataset. With the problems revealed, I corrected them using the audit.py file. <br />
This project was very challenging and very difficult to complete. I struggled a lot to understand how the audit.py and data.py files work and how I could adjust these files programmatically to clean the OpenStreetMap dataset. Other difficulties include understanding the structure of the OpenStreetMap dataset, using regular expressions to search for inconsistent words and performing SQL queries to answer some questions. This project is in its initial stage. There are lots of data to be standardized, cleaned, completed and corrected.